## Introduction

In this notebook, we will try to obtain the unique headers amongst all the csv files that we are going to merge. Then we will finally make a master comma seperated value file with list of headers that we obtain from this notebook

## Data Cleaning

### Import the necessary libaries 

In [2]:
import pandas as pd
import numpy as np
import glob
import csv
import re
import os

### Setting up parent directory and sub directory

In [3]:
parent_dir = "../UnifiedCSVTestData/"
filelist = []
dirs = []

def makefilelist(parent_dir):
    headers = []
    csv_headers = []
    subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
    filelist = []
    for dir in subject_dirs:
        csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
        for file in csv_files:
            filelist.append(file)
    
    return filelist, subject_dirs

### Read headers from CSV files

In [4]:
def readCSV(fileList, subject_dirs):
    master_csv_headers = []
    for filename in fileList:
        slash = filename.split("/")
        parts = slash[2]
        subparts = parts.split("_")
        CIK = subparts[0]
        report_type = subparts[1]
        subsubpart = subparts[2].split('-')
        report_year = subsubpart[1]    
        df = pd.read_csv(filename,engine='python')
        df['CIK'] = CIK
        df['Reporting Type'] = report_type
        df['Report Year'] = report_year
        df.to_csv(filename, encoding='utf-8', index=False)
        
        with open(filename, 'r') as f:
            d_reader = csv.DictReader(f)
            headers = d_reader.fieldnames
            for header in headers:
                master_csv_headers.append(header)
            
    return master_csv_headers

In [5]:
filelist, dirs = makefilelist(parent_dir)
csv_headers = readCSV(filelist, dirs)

Now we will only use the unqiue headers that we might require for our master CSV

In [6]:
def uniqueHeaders(csv_headers):
    return set(csv_headers)

def chomp(list1):
    list1 = [x.replace('\n', '') for x in list1]
    return list1

Checking our unique header values and taking a look on the kind of values we have

In [9]:
final_headers = uniqueHeaders(csv_headers)
print(len(uniqueHeaders(csv_headers)))
lol = chomp(final_headers)
print(len(lol))

1837
1837


### Checking for different instances of similar headers

Using this to test out the best conditions required to extract all the possible variants of each header in the unified CSV

In [14]:
counter = 0
for header in lol:
    if 'des' in lol:
        print(header)
        counter+=1
print ("Counter", counter)

Counter 0


We will usse the above function as a way to guage how many similar ways are there to report a specific header and then merge all the occurances together to the master csv when we are conducting the joining

### Normalizing the values present in the column

In [ ]:
def normalizeValues(filelist):
    for filename in filelist:
        with open(filename, 'r') as f:
            df = pd.read_csv(filename, engine='python', dtype=str)
            headers = list(df)
            for header in headers:
                if '000' in header:
                    df[header] = df[header].astype(str) + '000'
            print (df)
            print("Filename", filename)        
            df.to_csv(filename, index=False, header=True)

In [ ]:
normalizeValues(filelist)

### Generate conditions for each type of Header 

In this section, we will decide the final size of the unified CSV that we are going to generate for each time user searches for something. 

In [19]:
column_headers = ["CIK", "Reporting Type", "Reporting Year", "Counterparty", "Notional Amount", "Reference Entity/Obligation", "Fixed Rate", "Expiration Date", "Appreciation/Depreciation", "Upfront Payments Paid/Received", "Implied Credit Spread", "Buy/Sell Protection", "Description"]
unified_csv = pd.DataFrame()
CIK_csv = pd.DataFrame(columns=["CIK"])
Reporting_Type_csv = pd.DataFrame(columns=["Reporting Type"])
Reporting_Year_csv = pd.DataFrame(columns=["Reporting Year"])
Counterparty_csv = pd.DataFrame(columns=["Counterparty"])
Notional_Amount_csv = pd.DataFrame(columns=["Notional Amount"])
Reference_Entity_csv = pd.DataFrame(columns=["Reference Entity/Obligation"])
Fixed_Rate_csv = pd.DataFrame(columns=["Fixed Rate"])
Expiration_Date_csv = pd.DataFrame(columns=["Expiration Date"])
Appreciation_csv = pd.DataFrame(columns=["Appreciation/Depreciation"])
Upfront_Payments_csv = pd.DataFrame(columns=["Upfront Payments Paid/Received"])
Buy_Sell_csv = pd.DataFrame(columns=["Buy/Sell Protection"])
Description_csv = pd.DataFrame(columns=["Description"]) 

#### Joining the headers and merging them to a final CSV

In [21]:
for filename in filelist:
    #Add flgas to detect changes
    CIK_Flag = 0
    ReportingType_Flag = 0
    ReportingYear_Flag = 0
    Counterparty_Flag = 0
    NotionalAmount_Flag = 0
    ReferenceEntity_Flag = 0
    FixedRate_Flag = 0
    ExpirationDate_Flag = 0
    App_Dep_Flag = 0
    Upfront_Payments_Flag = 0
    Buy_Sell_Flag = 0
    Description_Flag = 0
    
    #Reading and performing basic data cleaning 
    df = pd.read_csv(filename, engine='python', dtype=str)
    df = df.replace(r'\n',' ', regex=True)
    df = df.replace(r'\t',' ', regex=True)
    #Putting the list of headers in a list
    headers = list(df)
    
    #Checking for CIK in the list if not append NaN for all rows
    for header in headers:
        #CIK NUMBER
        if 'cik' in header.lower():
            CIK_Flag = 1
            for i in df[header].iteritems():
                CIK_csv = CIK_csv.append({'CIK': i[1]}, ignore_index=True)
                
    if CIK_Flag == 0:
        for i in df[header].iteritems():
            CIK_csv = CIK_csv.append({'CIK': "NaN"}, ignore_index=True)
            
    #Checking for Reporting Type in the list if not append NaN for all rows        
    for header in headers:    
        if 'reporting' in header.lower():
            ReportingType_Flag = 1
            for i in df[header].iteritems():
                Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': i[1]}, ignore_index=True)
                
    if ReportingType_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Type_csv = Reporting_Type_csv.append({'Reporting Type': "NaN"}, ignore_index=True)
            
    #Checking for Reporting Year in the list if not append NaN for all rows   
    for header in headers:
        if 'year' in header.lower():
            ReportingYear_Flag = 1
            for i in df[header].iteritems():
                Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': i[1]}, ignore_index=True)
                
    if ReportingYear_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Reporting Year': "NaN"}, ignore_index=True)
    
    #Checking for Counterparty in the list if not append NaN for all rows
    for header in headers:    
        if 'counter' in header.lower() or 'party' in header.lower() or 'obligation' in header.lower():
            Counterparty_Flag = 1
            for i in df[header].iteritems():
                Counterparty_csv = Counterparty_csv.append({'Counterparty': i[1]}, ignore_index=True)
                
    if Counterparty_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Counterparty': "NaN"}, ignore_index=True)
    
    #Checking for Notional Amount in the list if not append NaN for all rows
    for header in headers:           
        if 'notional' in header.lower() or 'amount' in header.lower() or 'value' in header.lower():
            NotionalAmount_Flag = 1
            for i in df[header].iteritems():
                Notional_Amount_csv = Notional_Amount_csv.append({'Notional Amount': i[1]}, ignore_index=True)
                
    if NotionalAmount_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Notional Amount': "NaN"}, ignore_index=True)
                               
    #Checking for Reference Entity/Obligation in the list if not append NaN for all rows             
    for header in headers:              
        if 'reference' in header.lower() or ('entity' in header.lower() or 'obligation' in header.lower()):
            ReferenceEntity_Flag = 1
            for i in df[header].iteritems():
                
                Reference_Entity_csv = Reference_Entity_csv.append({'Reference Entity/Obligation': i[1]}, ignore_index=True)
    if ReferenceEntity_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Reference Entity/Obligation': "NaN"}, ignore_index=True)
    
    #Checking for Fixed Rate in the list if not append NaN for all rows
    for header in headers:
        if 'fixed' in header.lower() or 'rate' in header.lower():
            FixedRate_Flag = 1
            for i in df[header].iteritems():
                Reference_Entity_csv = Reference_Entity_csv.append({'Fixed Rate': i[1]}, ignore_index=True)
                
    if FixedRate_Flag == 0:
        for i in df[header].iteritems():
            Reporting_Year_csv = Reporting_Year_csv.append({'Fixed Rate': "NaN"}, ignore_index=True)  
            
     #Checking for Expiration Date in the list if not append NaN for all rows
    for header in headers:
        if 'date' in header.lower() or 'expiration' in header.lower() or 'termination' in header.lower() or 'maturity' in header.lower():
            ExpirationDate_Flag = 1
            for i in df[header].iteritems():
                Expiration_Date_csv = Expiration_Date_csv.append({'Expiration Date': i[1]}, ignore_index=True)
                
    if ExpirationDate_Flag == 0:
        for i in df[header].iteritems():
            Expiration_Date_csv = Expiration_Date_csv.append({'Expiration Date': "NaN"}, ignore_index=True)
    
    #Checking for Appreciation Depreciation in the list if not append NaN for all rows
    for header in headers:
        if 'appreciation' in header.lower() or 'depriciation' in header.lower():
            App_Dep_Flag = 1
            for i in df[header].iteritems():
                Appreciation_csv = Appreciation_csv.append({'Appreciation/Depreciation': i[1]}, ignore_index=True)
    
    if App_Dep_Flag == 0:
        for i in df[header].iteritems():
            Appreciation_csv = Appreciation_csv.append({'Appreciation/Depreciation': "NaN"}, ignore_index=True)
    
    #Checking for Upfront Payments Paid/Received in the list if not append NaN for all rows
    for header in headers:
        if 'upfront' in header.lower() or  ('payment' in header.lower() or 'premium' in header.lower()):
            Upfront_Payments_Flag = 1
            for i in df[header].iteritems():
                Upfront_Payments_csv = Upfront_Payments_csv.append({'Upfront Payments Paid/Received': i[1]}, ignore_index=True)
    
    if Upfront_Payments_Flag == 0:
        for i in df[header].iteritems():
            Upfront_Payments_csv = Upfront_Payments_csv.append({'Upfront Payments Paid/Received': "NaN"}, ignore_index=True)
    
    
    #Checking for Buy/Sell Protection in the list if not append NaN for all rows
    for header in headers:
        if 'buy' in header.lower() or  'sell' in header.lower() or 'protection' in header.lower():
            Buy_Sell_Flag = 0
            for i in df[header].iteritems():
                Buy_Sell_csv = Buy_Sell_csv.append({'Buy/Sell Protection': i[1]}, ignore_index=True)
    if Buy_Sell_Flag == 0:
        for i in df[header].iteritems():
            Buy_Sell_csv = Upfront_Payments_csv.append({'Buy/Sell Protection': "NaN"}, ignore_index=True)
    
    #Checking for Description in the list if not append NaN for all rows
    for header in headers:
        if 'description' in header.lower():
            Description_Flag = 1
            for i in df[header].iteritems():
                Description_csv = Description_csv.append({'Description': i[1]}, ignore_index=True)
    if Description_Flag == 0:
        for i in df[header].iteritems():
            Description_csv = Description_csv.append({'Description': "NaN"}, ignore_index=True)

print('Done')

KeyboardInterrupt: 

Let's check the final result

In [ ]:
Description_csv

In [ ]:
unified_csv = CIK_csv.join(Reporting_Type_csv)
unified_csv = unified_csv.join(Reporting_Year_csv)
unified_csv = unified_csv.join(Counterparty_csv)
unified_csv = unified_csv.join(Notional_Amount_csv)
unified_csv = unified_csv.join(Reference_Entity_csv)
unified_csv = unified_csv.join(Expiration_Date_csv)
unified_csv = unified_csv.join(Appreciation_csv)
unified_csv = unified_csv.join(Upfront_Payments_csv)
unified_csv = unified_csv.join(Buy_Sell_csv)
unified_csv = unified_csv.join(Description_csv)
unified_csv